In [16]:
import hopsworks
import pandas as pd
import tensorflow as tf
import os  # Agregar esta línea para importar os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [6]:
# Conectar con Hopsworks
project = hopsworks.login(api_key_value=os.getenv("HOPSWORKS_API_KEY"))
fs = project.get_feature_store()

# Cargar el dataset desde el Feature Store
imdb_fg = fs.get_feature_group(name="imdb_reviews_dl", version=1)
df = imdb_fg.read()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1047704
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (15.30s) 


In [20]:
# Separar características (X) y etiquetas (y)
X = df.drop(columns=['sentiment', 'id'])  # 'sentiment' es la etiqueta y 'id' es el índice
y = df['sentiment']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de Deep Learning
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64, input_length=X_train.shape[1]),  # Capa de embedding
    tf.keras.layers.LSTM(64, return_sequences=True),  # Primera capa LSTM
    tf.keras.layers.LSTM(32),  # Segunda capa LSTM
    tf.keras.layers.Dense(32, activation='relu'),  # Capa densa
    tf.keras.layers.Dense(1, activation='sigmoid')  # Capa de salida
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 64)           320000    
                                                                 
 lstm_8 (LSTM)               (None, 200, 64)           33024     
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dense_8 (Dense)             (None, 32)                1056      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 366529 (1.40 MB)
Trainable params: 366529 (1.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
500/500 

In [21]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")


250/250 - 7s - loss: 0.3371 - accuracy: 0.8675 - 7s/epoch - 29ms/step
Test Accuracy: 0.8675


In [22]:
# Guardar el modelo entrenado
model.save("modelo_imdb_dl.h5")


2024-09-21 03:10:17,741 WARNING: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [35]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

# Asegúrate de que el DataFrame tenga una columna 'review' con los textos
df = pd.read_csv("data/IMDB Dataset.csv")  # Ajusta el nombre del archivo y la ruta

# Parámetros del tokenizador
vocab_size = 33000
oov_tok = "<OOV>"
max_length = 200  # Ajusta la longitud máxima según tu modelo previo

# Inicializa el tokenizador
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# Entrenar el tokenizador en los textos de las reseñas
tokenizer.fit_on_texts(df['review'])

# Convertir los textos a secuencias numéricas
sequences = tokenizer.texts_to_sequences(df['review'])

# Rellenar (pad) las secuencias para que todas tengan la misma longitud
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Guardar el tokenizador para usarlo en la API y la notebook 3
joblib.dump(tokenizer, "tokenizer.pkl")

# Verificar el word_index del tokenizador
print(f"Word index length: {len(tokenizer.word_index)}")


Word index length: 124253


In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Definir el modelo LSTM
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Definir etiquetas (sentimientos) y preparar los datos de entrenamiento y validación
y = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0).values

# Separar en conjunto de entrenamiento y validación
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, y, test_size=0.2)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=64)

# Evaluar el modelo en el conjunto de validación
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

# Guardar el modelo entrenado
model.save("modelo_imdb_dl.h5")


Epoch 1/5
625/625 [==============================] - 82s 128ms/step - loss: 0.3556 - accuracy: 0.8386 - val_loss: 0.2738 - val_accuracy: 0.8856
Epoch 2/5
625/625 [==============================] - 80s 129ms/step - loss: 0.1795 - accuracy: 0.9322 - val_loss: 0.2906 - val_accuracy: 0.8836
Epoch 3/5
625/625 [==============================] - 78s 126ms/step - loss: 0.1092 - accuracy: 0.9619 - val_loss: 0.3461 - val_accuracy: 0.8813
Epoch 4/5
625/625 [==============================] - 79s 126ms/step - loss: 0.0733 - accuracy: 0.9748 - val_loss: 0.3709 - val_accuracy: 0.8740
Epoch 5/5
313/313 [==============================] - 8s 27ms/step - loss: 0.4702 - accuracy: 0.8733
Validation Accuracy: 0.8733000159263611
2024-09-21 04:09:49,904 WARNING: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.

